In [ ]:
DSCI 100 Individual Planning Stage

In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('cleanup.R')


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [8]:
read_csv("players.csv", show_col_types = FALSE)
read_csv("sessions.csv", show_col_types = FALSE)


experience,subscribe,hashedEmail,played_hours,name,gender,Age
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17
⋮,⋮,⋮,⋮,⋮,⋮,⋮
Amateur,FALSE,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db299bd4fedb06a46ad5bb,0.0,Dylan,Prefer not to say,57
Amateur,FALSE,f19e136ddde68f365afc860c725ccff54307dedd13968e896a9f890c40aea436,2.3,Harlow,Male,17
Pro,TRUE,d9473710057f7d42f36570f0be83817a4eea614029ff90cf50d8889cdd729d11,0.2,Ahmed,Other,NA


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
⋮,⋮,⋮,⋮,⋮
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,28/07/2024 15:36,28/07/2024 15:57,1.72218e+12,1.72218e+12
fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33cbb5e894a3867ca44d,25/07/2024 06:15,25/07/2024 06:22,1.72189e+12,1.72189e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,20/05/2024 02:26,20/05/2024 02:45,1.71617e+12,1.71617e+12


players.csv:
- Observations: 196
- Variables: 7

| Variable     | Type     	             | Missing Values |	Description                                              | 
| --------     | --------------------    | -------------- | -------------------------------------------------------- |
| experience   | Character (categorical) |	0	          | Player's self-reported experience level.                 |
| subscribe    | Logical (categorical)   |	0	          | Indicates whether the player has an active subscription. |
| hashedEmail  | Character (categorical) |	0	          | Anonymized, unique identifier for each player.           |
| played_hours | Double (numerical)      |	0	          | Total cumulative hours the player has played.            |
| name         | Character (categorical) |	0	          | The player's first name.                                 |
| gender	   | Character (categorical) |  0	          | The player's self-reported gender.                       |
| Age	       | Double (numerical)      |	2	          | The player's age.                                        |

Categorical variables:
- experience: 5 unique values.
- subscribe: 2 unique values.
- gender: 7 unique values.
- hashedEmail: 196 unique values.
- name: 196 unique values.

Observed issues:
- There are 2 missing values in the Age column.
- Age is stored as a float instead of an integer.
- Many users logged zero (or very little) hours played on the game, resulting in significant outliers.

Potential issues:
- Variables like Age, gender, and experience are self-reported and may be inaccurate.
- There are potentially multiple users who use the same hashedEmail indentifier, resulting in the loss of data.
- There are 196 observations, which is likely not the entire player base. If this is a sample, results may not be able to be generalized.

How the data was collected:
- A UBC computer science research group developed a game similar to Minecraft, and they kept track of specific statistics of each player, such as age, gender, and number of hours played.

sessions.csv:

- Observations: 1535
- Variables: 5


| Variable            | Type     	            | Missing Values |	Description                                                  | 
| ------------------- | ----------------------- | -------------- | ------------------------------------------------------------- |
| hashedEmail         | Character (categorical) |	0	         | Anonymized, unique identifier for each player.                |
| start_time          | Character (categorical) |	0	         | The start time of the session.                                |
| end_time            | Character (categorical) |	2	         | The end time of the session.                                  |
| original_start_time | Double (numerical)      |	0	         | Computer-readable timestamp of the start time of the session. |
| original_end_time   | Double (numerical)      |	2	         | Computer-readable timestamp of the end time of the session.   |                     

Observed issues:
- There are no units for original_start_time and original_end_time, and the meaning of each is quite unclear.
- Some players that appear in players.csv are not present in this dataset.
- 2 sessions have no end_time.

Potential issues:
- Many players may start a session but leave their screen temporarily, which would still count towards the duration of their session.
- There is no consistent timezone stated.

How the data was collected:
- A UBC computer science research group developed a game similar to Minecraft, and they kept track of the session activity for each player, matching a start and end time for each user.